In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
!pip install langchain-google-genai langchain

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 7.8 MB/s  0:00:00
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   --------------------- ------------------ 2.4/4.5 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 4.5/4.5 MB 11.8 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.5 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 12.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 10.8 MB/s  0:00:00

   ----------------------------------------  0/13 [filetype]
   ----------------------------------------  0/13 [filetype]
   --- ------------------------------------  1/13 [protobuf]
   --- -------

In [11]:
from langchain.chat_models import init_chat_model

In [14]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [18]:
llm_response = llm.invoke("what is langchain?")
llm_response

AIMessage(content="LangChain is a framework designed to simplify the development of applications powered by large language models (LLMs). Think of it as a toolkit that helps you connect LLMs to other data sources and tools, allowing you to build more complex and powerful applications.\n\nHere's a breakdown of what LangChain is and what it does:\n\n**Key Concepts and Components:**\n\n* **LLMs (Large Language Models):** These are the core engines of LangChain applications. Examples include GPT-3, GPT-4, PaLM, and open-source models like Llama 2. LangChain provides interfaces and abstractions to interact with these models.\n\n* **Chains:** These are sequences of calls, whether to an LLM or a utility.  They represent a structured way to combine multiple components to achieve a specific goal.  For example, a chain might first retrieve relevant documents, then use an LLM to answer a question based on those documents.\n\n* **Indexes:**  LangChain provides tools for indexing and retrieving dat

In [23]:
llm_response.content

"LangChain is a framework designed to simplify the development of applications powered by large language models (LLMs). Think of it as a toolkit that helps you connect LLMs to other data sources and tools, allowing you to build more complex and powerful applications.\n\nHere's a breakdown of what LangChain is and what it does:\n\n**Key Concepts and Components:**\n\n* **LLMs (Large Language Models):** These are the core engines of LangChain applications. Examples include GPT-3, GPT-4, PaLM, and open-source models like Llama 2. LangChain provides interfaces and abstractions to interact with these models.\n\n* **Chains:** These are sequences of calls, whether to an LLM or a utility.  They represent a structured way to combine multiple components to achieve a specific goal.  For example, a chain might first retrieve relevant documents, then use an LLM to answer a question based on those documents.\n\n* **Indexes:**  LangChain provides tools for indexing and retrieving data, making it easie

In [22]:
llm_response.response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.0-flash',
 'safety_ratings': []}

In [34]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [35]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [36]:
message = {"role": "user", "content" : "Who walked on the moon for the first time? Print only the name."}
response = graph.invoke({"messages": [message]})

In [38]:
response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name.', additional_kwargs={}, response_metadata={}, id='1ace160c-5142-4c2d-9406-64faf5a26783'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0a1e74f6-2174-4d84-8007-d9defb7c59f2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})]

In [43]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state = {"messages": [{"role": "user", "content": in_message}]} 
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Neil Armstrong was the first person to land on the moon.
Bot: Neil Armstrong landed on the moon in **1969**.


In [39]:
message = {"role": "user", "content" : "What is the latest price of APPL stock?"}
response = graph.invoke({"messages": [message]})
response["messages"]

[HumanMessage(content='What is the latest price of APPL stock?', additional_kwargs={}, response_metadata={}, id='33a169aa-a8cd-41a8-a6a3-e16f457799bf'),
 AIMessage(content='Unfortunately, I cannot give you the exact, real-time price of AAPL stock. Stock prices change constantly throughout the day.\n\nHowever, here\'s how you can find the most up-to-date information:\n\n*   **Reliable Online Financial Websites:** Go to a reputable financial website like Google Finance, Yahoo Finance, Bloomberg, or MarketWatch. Search for "AAPL" (Apple\'s stock ticker symbol) on any of these sites. They will provide real-time stock quotes, charts, and other relevant information.\n*   **Brokerage Account:** If you have a brokerage account (e.g., Fidelity, Schwab, Robinhood), log in to your account. The stock price will be displayed there.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safe